In [4]:
from sklearn.cluster import KMeans
import json
import psycopg2
from datetime import datetime, date
import math
import numpy as np

In [ ]:
def calculate_distance(A,B):
  A_x = A[1]
  A_y = A[2]
  B_x = B[1]
  B_y = B[2]
  dist = math.sqrt((A_x - B_x)**2 + (A_y - B_y)**2)
  return dist

In [ ]:
def make_dist_matrix(location_list):
  matrix_size = len(location_list)
  #initialize empty n*n matrix
  dist_matrix = [([0] * matrix_size) for _ in range(matrix_size)]
  for i in range(matrix_size):
    for j in range(i+1,matrix_size):
      dist_matrix[i][j] = calculate_distance(location_list[i],location_list[j])
      dist_matrix[j][i] = dist_matrix[i][j]
  return dist_matrix

##변경전 : 단순 거리 최적화 순대로 클러스터 지정

In [ ]:
def find_optimal(dist_matrix,day):
  print("dist matrix : ",dist_matrix)
  total_dist = [0]*day
  visit_result = [[] for _ in range(day)]
  for i in range(day):
    start = i
    visit = [False] * day
    visit[i] = True    
    now = i
    min_dist = 100
    min_idx = 999
    visit_result[i].append(now)
    while False in visit: #모든 클러스터를 방문할때까지
      for idx, dist in enumerate(dist_matrix[now]):
        if visit[idx] == True:
          continue
        else:
          if dist < min_dist:
            min_idx = idx
            min_dist = dist
      print("min idx, min_dist : ",min_idx,min_dist)
      
      total_dist[i] += min_dist
      now = min_idx
      visit[now] = True
      visit_result[i].append(now)
      min_dist=999
    print("total_dist : ",total_dist[i])

  shortest_cluster = np.array(total_dist).argsort()[0]
  return visit_result[shortest_cluster]

##1일차는 공항이랑 가장 가까운곳으로 배정하기

In [13]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [10]:
def load_places_location(db, place_list):
  cursor = db.cursor()
  place_list = str(tuple(place_list))
  sql = f"select place_id, latitude, longitude from places where place_id in {place_list}"
  cursor.execute(sql)
  result = cursor.fetchall()
  result = list(map(list,result))
  return result

In [7]:
def calculate_distance(A,B):
  A_x = A[1]
  A_y = A[2]
  B_x = B[1]
  B_y = B[2]
  dist = math.sqrt((A_x - B_x)**2 + (A_y - B_y)**2)
  return dist

In [8]:
def make_dist_matrix(location_list):
  matrix_size = len(location_list)
  #initialize empty n*n matrix
  dist_matrix = [([0] * matrix_size) for _ in range(matrix_size)]
  for i in range(matrix_size):
    for j in range(i+1,matrix_size):
      dist_matrix[i][j] = calculate_distance(location_list[i],location_list[j])
      dist_matrix[j][i] = dist_matrix[i][j]
  return dist_matrix

In [6]:
JEJU_AIRPORT = (0,33.5059364, 126.4959513)

In [33]:
#sex method
def find_cluster_near_airport(cluster_info,airport):
  min = 9999
  cluster_num = -1 #공항에서 가장 가까운 클러스터 번호
  for val in cluster_info:
    new_dist = calculate_distance(val,airport)
    
    if min > new_dist:
      print("min : ",new_dist)
      print("cluster_info: ",val)
      min = new_dist
      cluster_num = val[0]
  return cluster_num

In [ ]:
find_cluster_near_airport()

In [22]:
def find_optimal(dist_matrix,day,first_day):
  print("dist matrix : ",dist_matrix)
  total_dist = [0]*day
  visit_result = []
  
  #1일차가 될 클러스터는 공항에서 가장 가까운곳으로 선정. 다음 2일차, 3일차는 now에서 가장 가까운곳으로 선정
  
  start = first_day
  visit = [False] * day
  visit[start] = True    
  now = start
  visit_result.append(now)
  min_dist = 9999
  min_idx = -1
  #visit_result[i].append(now)
  while False in visit: #모든 클러스터를 방문할때까지
    #인접한 모든 클러스터를 방문후에 가장 가까운 클러스터 택 1
    for idx, dist in enumerate(dist_matrix[now]):
      if visit[idx] == True:
        continue
      else:
        if dist < min_dist:
          min_idx = idx
          min_dist = dist
    print("min idx, min_dist : ",min_idx,min_dist)

    now = min_idx
    visit[now] = True
    visit_result.append(now)
    min_dist=999
   
  return visit_result

In [23]:
def optimize_course(place_info,day):

  save_id = []  
  result = [[] for _ in range(day)]
  for i in place_info:
    save_id.append(i[0])
  #id는 클러스터링에 무관하게 만들기
  for i in place_info:
    i[0] = 0
    
  kmeans = KMeans(n_clusters=day,random_state=0)
  kmeans = kmeans.fit(place_info)
  
  cluster_center = kmeans.cluster_centers_
  for i in range(day):
    cluster_center[i][0] = i
  print(cluster_center)
  
  #각 클러스터별 중심좌표를 이용해 n*n matrix 만들기
  dist_matrix = make_dist_matrix(cluster_center)
  #첫 날은 공항에서 가장 가까운 곳으로
  first_day =  int(find_cluster_near_airport(cluster_center,JEJU_AIRPORT))
  print("first day : ",first_day)
  #클러스터간 방문 순서는 거리 기준으로 최적화하기
  optimal_cluster_order = find_optimal(dist_matrix,day,first_day)

  for idx, label in enumerate(kmeans.labels_):
    now = save_id[idx]
    result[label].append(int(now))
  print("\nresult : ",result)
  #이제 원래의 Idx와 중심좌표를 기준으로 
  new_result = []
  for i in optimal_cluster_order:
    new_result.append(result[i])
  return new_result

In [31]:
db = load_db()
place_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
p_info = load_places_location(db,place_list)
p_info

[[1, Decimal('33.4077504'), Decimal('126.6424787')],
 [2, Decimal('33.5194929'), Decimal('126.9510302')],
 [3, Decimal('33.4621574'), Decimal('126.9363164')],
 [4, Decimal('33.4917178'), Decimal('126.8112799')],
 [5, Decimal('33.2447169'), Decimal('126.5595512')],
 [6, Decimal('33.5283774'), Decimal('126.7716157')],
 [7, Decimal('33.3673209'), Decimal('126.3570070')],
 [8, Decimal('33.3937480'), Decimal('126.2394319')],
 [9, Decimal('33.4302205'), Decimal('126.9280527')],
 [10, Decimal('33.2901402'), Decimal('126.3683652')],
 [11, Decimal('33.4568773'), Decimal('126.7142801')],
 [12, Decimal('33.5437098'), Decimal('126.6687041')],
 [13, Decimal('33.5551082'), Decimal('126.7986579')],
 [14, Decimal('33.3926876'), Decimal('126.4948419')],
 [15, Decimal('33.2521457'), Decimal('126.6230070')],
 [16, Decimal('33.2379927'), Decimal('126.4260485')],
 [17, Decimal('33.2447480'), Decimal('126.5730501')],
 [18, Decimal('33.4553485'), Decimal('126.6688528')],
 [19, Decimal('33.4590482'), Decimal(

In [32]:
optimize_course(p_info,5)

[[  0.          33.49126991 126.72512417]
 [  1.          33.4263981  126.27497935]
 [  2.          33.24720353 126.58520277]
 [  3.          33.4706236  126.93846643]
 [  4.          33.32203535 126.41156565]]
min :  0.229641700921294
cluster_info:  [  0.          33.49126991 126.72512417]
min :  0.20233767349909487
cluster_info:  [  4.          33.32203535 126.41156565]
first day :  4
dist matrix :  [[0, 0.4547952424417924, 0.281329695947191, 0.21433896287941245, 0.35631346327279106], [0.4547952424417924, 0, 0.3582586509369993, 0.6649594007158844, 0.17189357444434558], [0.281329695947191, 0.3582586509369993, 0, 0.41798533990573694, 0.18907577599985634], [0.21433896287941245, 0.6649594007158844, 0.41798533990573694, 0, 0.5474512795814274], [0.35631346327279106, 0.17189357444434558, 0.18907577599985634, 0.5474512795814274, 0]]
min idx, min_dist :  1 0.17189357444434558
min idx, min_dist :  2 0.3582586509369993
min idx, min_dist :  0 0.281329695947191
min idx, min_dist :  3 0.2143389628

[[7, 10, 14, 16], [8, 19], [5, 15, 17], [1, 4, 6, 11, 12, 13, 18], [2, 3, 9]]